# Reinforcement Fine-Tuning with OpenR1-Math-220k Dataset

This notebook demonstrates how to fine-tune language models using **Reinforcement Fine-Tuning (RFT)** with the OpenR1-Math-220k dataset - a collection of 220,000 advanced mathematical reasoning problems with verified step-by-step solutions.

---

##  Agenda

1. **Dataset Overview & Preparation**
2. **Environment Setup**
3. **Data Validation & Exploration**
4. **Base Model Evaluation**
5. **Mathematical Grader Setup**
6. **RFT Training Configuration**
7. **Launch Fine-Tuning Job**
8. **Monitor Training Progress**
9. **Deploy & Test Fine-Tuned Model**
10. **Evaluation & Comparison**

---

## 1 Dataset Overview

**OpenR1-Math-220k** contains:
- **220,000** mathematical reasoning problems
- **2-4 verified reasoning traces** per problem
- Problems from college-level and competition mathematics
- Solutions with **detailed step-by-step reasoning**
- Final answers in `\boxed{}` format
- **88%** verified using Math Verify, **12%** using Llama-3.3-70B-Instruct

**Problem Domains:**
- Algebra and polynomial factorization
- Calculus and optimization
- Probability theory
- Geometry and trigonometry
- Number theory and combinatorics
- Linear algebra
- Complex numbers and abstract mathematics

**Why RFT?**
- Multiple valid solution paths exist
- Correctness can be verified automatically
- Model learns from quality reasoning patterns
- Better than SFT for multi-path reasoning tasks

## 2 Environment Setup

In [ ]:
# Install required packages (if not already installed)
# !pip install -r requirements.txt

In [ ]:
import os
import json
import re
from pathlib import Path
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
import random

# Load environment variables
load_dotenv()

print(" Imports successful!")

In [ ]:
# Azure AI Configuration
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group = os.getenv("AZURE_RESOURCE_GROUP")
aoai_account = os.getenv("AZURE_AOAI_ACCOUNT")
model_name = os.getenv("MODEL_NAME", "gpt-4o")  # Base model for RFT

# Azure OpenAI for evaluation
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
deployment_name = os.getenv("DEPLOYMENT_NAME")

print("Configuration loaded:")
print(f"  Project Endpoint: {project_endpoint}")
print(f"  Base Model: {model_name}")
print(f"  Deployment: {deployment_name}")

In [ ]:
# Initialize Azure AI Project Client
credential = DefaultAzureCredential()
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group
)

print(" Azure AI Project Client initialized")

## 3 Data Validation & Exploration

Before training, let's examine our prepared training data.

In [ ]:
# Check if JSONL files exist
training_file = Path("training.jsonl")
validation_file = Path("validation.jsonl")

if not training_file.exists() or not validation_file.exists():
    print(" Training or validation files not found!")
    print("\n Please run the data preparation script first:")
    print("   python scripts/prepare_data.py --input_dir ./training_data --output_dir ./")
else:
    # Count examples
    with open(training_file, 'r', encoding='utf-8') as f:
        train_count = sum(1 for _ in f)
    
    with open(validation_file, 'r', encoding='utf-8') as f:
        val_count = sum(1 for _ in f)
    
    print(f" Dataset files found:")
    print(f"   Training examples: {train_count:,}")
    print(f"   Validation examples: {val_count:,}")
    print(f"   Total: {train_count + val_count:,}")

In [ ]:
# Explore sample training examples
print(" Sample Training Examples:\n" + "="*80)

with open(training_file, 'r', encoding='utf-8') as f:
    # Show 2 random examples
    lines = f.readlines()
    samples = random.sample(lines, min(2, len(lines)))
    
    for i, line in enumerate(samples, 1):
        example = json.loads(line)
        messages = example['messages']
        
        print(f"\n Example {i}:")
        print(f"\nProblem:\n{messages[1]['content'][:300]}...")
        print(f"\nSolution Preview:\n{messages[2]['content'][:500]}...")
        print(f"\nSolution Length: {len(messages[2]['content'])} characters")
        print("\n" + "-"*80)

## 4 Base Model Evaluation

Test the base model's performance on a few mathematical problems before fine-tuning.

In [ ]:
# Initialize chat client for evaluation
chat_client = ChatCompletionsClient(
    endpoint=azure_openai_endpoint,
    credential=AzureKeyCredential(azure_openai_key)
)

# System prompt for mathematical reasoning
math_system_prompt = (
    "You are a mathematical reasoning expert. Solve problems with detailed "
    "step-by-step thinking and provide final answers in \\boxed{} format. "
    "Show all intermediate calculations and explain your reasoning clearly."
)

In [ ]:
# Test base model on a sample problem
def test_base_model(problem, max_tokens=4000):
    """Test base model on a mathematical problem."""
    response = chat_client.complete(
        model=deployment_name,
        messages=[
            SystemMessage(content=math_system_prompt),
            UserMessage(content=problem)
        ],
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content

# Get a test problem
with open(validation_file, 'r', encoding='utf-8') as f:
    test_example = json.loads(f.readline())
    test_problem = test_example['messages'][1]['content']
    ground_truth = test_example['messages'][2]['content']

print(" Testing Base Model:\n" + "="*80)
print(f"\nProblem:\n{test_problem}\n")

base_response = test_base_model(test_problem)

print(f"\nBase Model Response:\n{base_response}\n")
print("="*80)
print(f"\nGround Truth (first 500 chars):\n{ground_truth[:500]}...")

## 5 Mathematical Grader Setup

For RFT, we need a grading function that evaluates the quality of mathematical reasoning. This grader will:
1. Check if the answer is in the correct `\boxed{}` format
2. Extract and compare the final answer
3. Evaluate reasoning quality (length, structure, completeness)
4. Assign a reward score (0.0 to 1.0)

In [ ]:
def extract_boxed_answer(text):
    """Extract answer from \boxed{} notation."""
    pattern = r'\\boxed\{([^}]+)\}'
    matches = re.findall(pattern, text)
    return matches[-1] if matches else None

def grade_mathematical_solution(solution, ground_truth=None):
    """
    Grade a mathematical solution based on:
    - Format compliance (has \boxed{} answer)
    - Reasoning quality (length, structure)
    - Answer correctness (if ground truth provided)
    
    Returns score between 0.0 and 1.0
    """
    score = 0.0
    
    # Check for boxed answer (30% of score)
    predicted_answer = extract_boxed_answer(solution)
    if predicted_answer:
        score += 0.3
    else:
        return 0.1  # Minimal score if no answer provided
    
    # Check reasoning length (20% of score)
    # Good solutions are typically 1000-8000 tokens
    solution_length = len(solution)
    if solution_length > 500:
        score += 0.2
    elif solution_length > 200:
        score += 0.1
    
    # Check for step-by-step reasoning indicators (20% of score)
    step_indicators = ['step', 'first', 'next', 'then', 'therefore', 'thus', 'hence']
    step_count = sum(1 for indicator in step_indicators if indicator.lower() in solution.lower())
    if step_count >= 3:
        score += 0.2
    elif step_count >= 1:
        score += 0.1
    
    # Check answer correctness if ground truth provided (30% of score)
    if ground_truth:
        gt_answer = extract_boxed_answer(ground_truth)
        if gt_answer and predicted_answer:
            # Normalize answers for comparison
            pred_normalized = predicted_answer.strip().lower().replace(' ', '')
            gt_normalized = gt_answer.strip().lower().replace(' ', '')
            
            if pred_normalized == gt_normalized:
                score += 0.3
            elif pred_normalized in gt_normalized or gt_normalized in pred_normalized:
                score += 0.15
    
    return min(score, 1.0)

# Test the grader
test_solution = """Let's solve this step by step.
First, we analyze the problem.
Next, we apply the formula.
Therefore, the answer is \\boxed{42}."""

print(f"Test grader score: {grade_mathematical_solution(test_solution):.2f}")
print(" Grading function ready")

## 6 Upload Training Data

Upload the training and validation datasets to Azure AI.

In [ ]:
# Upload training file
print(" Uploading training data...")
with open(training_file, "rb") as f:
    train_data = project_client.upload_file(f)
print(f" Training data uploaded: {train_data.id}")

# Upload validation file
print(" Uploading validation data...")
with open(validation_file, "rb") as f:
    val_data = project_client.upload_file(f)
print(f" Validation data uploaded: {val_data.id}")

## 7 Configure RFT Training

Set up the fine-tuning configuration for Reinforcement Fine-Tuning.

In [ ]:
# RFT Training Configuration
rft_config = {
    "model": model_name,
    "training_file": train_data.id,
    "validation_file": val_data.id,
    "hyperparameters": {
        "n_epochs": 2,  # 2-3 epochs for mathematical reasoning
        "batch_size": 1,  # Small batch for long reasoning chains
        "learning_rate_multiplier": 0.5  # Conservative LR to preserve reasoning
    },
    "suffix": "math-reasoning-rft",  # Model name suffix
}

print(" RFT Configuration:")
print(f"  Model: {rft_config['model']}")
print(f"  Epochs: {rft_config['hyperparameters']['n_epochs']}")
print(f"  Batch Size: {rft_config['hyperparameters']['batch_size']}")
print(f"  Learning Rate Multiplier: {rft_config['hyperparameters']['learning_rate_multiplier']}")
print(f"  Model Suffix: {rft_config['suffix']}")

## 8 Launch Fine-Tuning Job

In [ ]:
# Create fine-tuning job
print(" Launching RFT fine-tuning job...")

fine_tune_job = project_client.inference.create_fine_tuning_job(
    model=rft_config["model"],
    training_file=rft_config["training_file"],
    validation_file=rft_config["validation_file"],
    hyperparameters=rft_config["hyperparameters"],
    suffix=rft_config["suffix"]
)

job_id = fine_tune_job.id
print(f"\n Fine-tuning job created: {job_id}")
print(f"   Status: {fine_tune_job.status}")
print(f"   Model: {fine_tune_job.model}")

## 9 Monitor Training Progress

In [ ]:
import time

# Monitor job status
print(" Monitoring training progress...\n")

while True:
    job_status = project_client.inference.get_fine_tuning_job(job_id)
    status = job_status.status
    
    print(f"Status: {status} - {time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    if status in ["succeeded", "failed", "cancelled"]:
        break
    
    time.sleep(60)  # Check every minute

if status == "succeeded":
    print(f"\n Fine-tuning completed successfully!")
    print(f"   Fine-tuned model: {job_status.fine_tuned_model}")
    fine_tuned_model = job_status.fine_tuned_model
else:
    print(f"\n Fine-tuning {status}")
    if hasattr(job_status, 'error'):
        print(f"   Error: {job_status.error}")

##  Deploy Fine-Tuned Model

In [ ]:
# Deploy the fine-tuned model
print(" Deploying fine-tuned model...")

deployment = project_client.inference.create_deployment(
    model=fine_tuned_model,
    name=f"math-reasoning-{int(time.time())}"
)

deployment_name_ft = deployment.name
print(f"\n Model deployed: {deployment_name_ft}")
print(f"   Endpoint: {deployment.endpoint}")

## 11 Test Fine-Tuned Model

Compare the fine-tuned model's performance with the base model.

In [ ]:
# Test fine-tuned model on the same problem
def test_finetuned_model(problem, deployment_name, max_tokens=4000):
    """Test fine-tuned model on a mathematical problem."""
    response = chat_client.complete(
        model=deployment_name,
        messages=[
            SystemMessage(content=math_system_prompt),
            UserMessage(content=problem)
        ],
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content

print(" Testing Fine-Tuned Model:\n" + "="*80)
print(f"\nProblem:\n{test_problem}\n")

finetuned_response = test_finetuned_model(test_problem, deployment_name_ft)

print(f"\nFine-Tuned Model Response:\n{finetuned_response}\n")
print("="*80)

## 12 Evaluation & Comparison

Evaluate both models on multiple test problems and compare performance.

In [ ]:
# Evaluate on multiple test problems
num_test_problems = 5

print(f" Evaluating models on {num_test_problems} problems...\n")

base_scores = []
ft_scores = []

with open(validation_file, 'r', encoding='utf-8') as f:
    test_examples = [json.loads(line) for line in list(f)[:num_test_problems]]

for i, example in enumerate(test_examples, 1):
    problem = example['messages'][1]['content']
    ground_truth = example['messages'][2]['content']
    
    print(f"\n Problem {i}/{num_test_problems}")
    print(f"Problem: {problem[:100]}...")
    
    # Test base model
    base_solution = test_base_model(problem)
    base_score = grade_mathematical_solution(base_solution, ground_truth)
    base_scores.append(base_score)
    print(f"  Base model score: {base_score:.2f}")
    
    # Test fine-tuned model
    ft_solution = test_finetuned_model(problem, deployment_name_ft)
    ft_score = grade_mathematical_solution(ft_solution, ground_truth)
    ft_scores.append(ft_score)
    print(f"  Fine-tuned model score: {ft_score:.2f}")
    print(f"  Improvement: {(ft_score - base_score):.2f}")

# Summary
avg_base = sum(base_scores) / len(base_scores)
avg_ft = sum(ft_scores) / len(ft_scores)

print("\n" + "="*80)
print(" EVALUATION SUMMARY")
print("="*80)
print(f"Average Base Model Score: {avg_base:.3f}")
print(f"Average Fine-Tuned Model Score: {avg_ft:.3f}")
print(f"Average Improvement: {(avg_ft - avg_base):.3f} ({((avg_ft - avg_base) / avg_base * 100):.1f}%)")
print("="*80)

##  Key Takeaways

After completing this RFT fine-tuning cookbook, you should observe:

 **Improved Reasoning Quality**
- More detailed step-by-step explanations
- Better structured mathematical arguments
- Clearer intermediate calculations

 **Better Format Compliance**
- Consistent use of `\boxed{}` for final answers
- Proper mathematical notation
- Well-organized solution structure

 **Enhanced Problem-Solving**
- Higher accuracy on complex problems
- Better handling of multi-step reasoning
- Improved performance across different mathematical domains

 **RFT Advantages**
- Model learns from multiple solution paths
- Reward-based learning encourages quality reasoning
- Better generalization than supervised fine-tuning

---

##  Cleanup (Optional)

Remember to delete resources when you're done to avoid charges:
- Fine-tuning jobs
- Deployed models
- Uploaded files

In [ ]:
# Cleanup code (uncomment to use)
# print(" Cleaning up resources...")

# # Delete deployment
# project_client.inference.delete_deployment(deployment_name_ft)
# print(f" Deleted deployment: {deployment_name_ft}")

# # Delete uploaded files
# project_client.delete_file(train_data.id)
# project_client.delete_file(val_data.id)
# print(" Deleted uploaded files")

# print("\n Cleanup complete!")